"""
This script is used to caclulate the Event Mean Concentration (EMC).
The inputs are .csv files containing concentration and flow after linear interpolation.
"""

In [3]:
import pandas as pd
import numpy as np
from utils.concentration import rainfall_events, emc_cal, conc_interpolate, event_emc
import datetime

# read the discrete storm events
# Read daily loads and flow
# Read hourly loads and flow
from common_settings import obspath, outpath, events_name, \
    obs_events, day_load_flow, hour_load_flow, conct_name, modpath, mod_load_flow

In [4]:
from utils.concentration import cumulative_lq, excel_save
from utils.signatures import update_cumul_df, load_flow_loc

## Produce the event mean concentration of obs and mod

In [3]:
# Calculate EMC for low-frequency data
cols = [col for col in day_load_flow.columns if ('Load' in col) or ('Flow(ML)' in col)]
index_range1 = [1, 38]
index_range2 = [38, obs_events.shape[0]+1]

In [4]:
obs_events = event_emc(obs_events, day_load_flow, index_range1, cols[0], cols[1], 
    time_scale='d', multiplier=1e3)

In [5]:
# Calculate EMC for high-frequency data
cols = [col for col in hour_load_flow.columns if ('Load' in col) or ('ML' in col)]
index_range2 = [38, obs_events.shape[0]+1]
loads_col = cols[1]; flow_col = cols[0]
obs_events = event_emc(obs_events, hour_load_flow, index_range2, loads_col, flow_col, 
    time_scale='h', multiplier=1)

In [7]:
obs_events.to_csv(outpath + events_name, index='ID')

In [8]:
# read the discrete storm events
filename = 'mod_storm_event_common.csv'
events = rainfall_events(f'{modpath}{filename}')
# Calculate EMC for modeling data
cols = [col for col in mod_load_flow.columns if ('Load' in col) or ('ML' in col)]
index_range = [1, events.shape[0]+1]
loads_col = cols[0]; flow_col = cols[1]

events = event_emc(events, mod_load_flow, index_range, loads_col, flow_col, 
    time_scale='d', multiplier=1)
events.dropna(axis=0, inplace=True)
events.to_csv(f'{outpath}DIN_{filename}', index='ID')

## Produce the Normalized cumulative ratio of loads and flow 

### calculate the daily data for double mass plot (Q-L)

In [34]:
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2020)]
# time_ranges = obs_events.loc[:, ['start', 'end']].values
double_mass_ratio = {}
annual_total = pd.DataFrame(columns=day_load_flow.columns)

In [35]:
for ii in range(0, len(time_ranges)-2):
# for ii in range(index_range1[0]-1, index_range1[1]-1):
    df_temp = load_flow_loc(time_ranges[ii], day_load_flow, timestep='d')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, 0], df_temp.values[:, -2])
    double_mass_ratio[f'obs_year_{ii}'] = df_temp
    annual_total.loc[time_ranges[ii][0][0:4]] = df_temp.sum(axis=0)
annual_total.loc['ave'] = annual_total.mean(axis=0)

In [37]:
# save outputs into one excel
annual_total.to_csv(outpath+'obs_annual_sum.csv')
fn = outpath +'obs_year_cumulative_ratio_day.xlsx'
excel_save(double_mass_ratio, fn, True)

### calculate the hourly data for double mass plot (Q-L)

In [15]:
double_mass_ratio = {}
for ii in range(index_range2[0]-1, index_range2[1]-1):
# for ii in range(9, len(time_ranges)):
    df_temp = load_flow_loc(time_ranges[ii], hour_load_flow, timestep='h')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, -1], df_temp.values[:, 0])
    double_mass_ratio[f'obs_storm_{ii}'] = df_temp

In [16]:
# save outputs into one excel
fn = outpath +'obs_storm_cumulative_ratio_hour.xlsx'
excel_save(double_mass_ratio, fn)

### calculate the modeling data for double mass plot (Q-L)

In [39]:
modpath = '../data/mod/'
annual_total = pd.DataFrame(columns=mod_load_flow.columns)
# filename = 'storm_event.csv'
# mod_events = rainfall_events(f'{modpath}{filename}')

In [40]:
# Calculate EMC for modeling data
cols = [col for col in mod_load_flow.columns if ('Load' in col) or ('ML' in col)]
# index_range = [1, mod_events.shape[0]]
loads_col = cols[0]; flow_col = cols[1]

In [41]:
double_mass_ratio = {}
time_ranges = [[f'{year}-07-01', f'{year+1}-06-30'] for year in range(2009, 2018)]
# time_ranges = mod_events.loc[:, ['start', 'end']].values
# for ii in range(index_range[0], index_range[1]):
for ii in range(len(time_ranges)):
    df_temp = load_flow_loc(time_ranges[ii], mod_load_flow, timestep='d')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, 0], df_temp.values[:, -1])
    double_mass_ratio[f'mod_storm_{ii}'] = df_temp
    annual_total.loc[time_ranges[ii][0][0:4]] = df_temp.sum(axis=0)
annual_total.loc['ave'] = annual_total.mean(axis=0)

In [42]:
# save results
annual_total.to_csv(outpath+'mod_annual_sum.csv')

fn = outpath +'mod_year_cumulative_ratio_day.xlsx'
excel_save(double_mass_ratio, fn)

## Calculate event load coefficients

### Event loads for obs

In [23]:
obs_event_fn = 'obs_storm_event_common'
obs_events = pd.read_csv(f'{outpath}{obs_event_fn}.csv', index_col = 'ID') 
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in [2009, 2010, 2011, 2012, 2013, 2014, 2018, 2019]]
year_loads = {}

In [24]:
# for each year, calculate the yearly loads 
# obs daily data
for tt in time_ranges[0:-2]:
    df = load_flow_loc(tt, day_load_flow, timestep='d')
    year_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  
    
# obs hourly data
for tt in time_ranges[-2:]:
    df = load_flow_loc(tt, hour_load_flow, timestep='h')
    year_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  

In [25]:
# The event load coefficients
for ii in range(1, index_range1[1]):
    df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, day_load_flow, timestep='d')
    ymd= pd.to_datetime(obs_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year - 1)]
    else:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year)]
        
for ii in range(index_range2[0], index_range2[1]):
    df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, hour_load_flow, timestep='h')
    ymd= pd.to_datetime(obs_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year-1)]
    else:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year)]

In [27]:
obs_events.to_csv(f'{outpath}{obs_event_fn}.csv')

### Event loads for mod

In [28]:
mod_event_fn = 'DIN_mod_storm_event_common'
mod_events = pd.read_csv(f'{outpath}{mod_event_fn}.csv', index_col = 'ID') 
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2014)]

In [29]:
# for each year, calculate the yearly loads 
mod_loads = {}
# mod daily data
for tt in time_ranges:
    df = load_flow_loc(tt, mod_load_flow, timestep='d')
    mod_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  

In [30]:
# The event load coefficients
for ii in mod_events.index:
    df_event = load_flow_loc(mod_events.loc[ii, 'start':'end'].values, mod_load_flow, timestep='d')
    ymd= pd.to_datetime(mod_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        mod_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / mod_loads[str(year - 1)]
    else:
        mod_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / mod_loads[str(year)]

In [31]:
mod_events.to_csv(f'{outpath}{mod_event_fn}.csv')

## Calculate the peaktime difference between flow and loads

### Mod results

In [32]:
mod_event_fn = 'DIN_mod_storm_event_common'
mod_events = pd.read_csv(f'{outpath}{mod_event_fn}.csv', index_col = 'ID') 

In [33]:
# find the peak time of loads
clabel, llabel, qlabel = ['Downstream Flow Concentration (mg.L^-1)', 'Loads (kg)', 'Flow_cumecs (ML.day^-1)']
for ii in mod_events.index:
    df_event = load_flow_loc(mod_events.loc[ii, 'start':'end'].values, mod_load_flow, timestep='d')
    peaktime_load = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index
    peaktime_conc = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]
    
    mod_events.loc[ii, 'peaktime_load'] = peaktime_load
    mod_events.loc[ii, 'peaktime_conc'] = peaktime_conc
    mod_events.loc[ii, 'peakflow'] = df_event.loc[:, qlabel].max()
    mod_events.loc[ii, 'peaktime'] = df_event[df_event.loc[:, qlabel]==df_event.loc[:, qlabel].max()].index
    mod_events.loc[ii, 'peak_flow_time'] = df_event[df_event.loc[:, qlabel]==df_event.loc[:, qlabel].max()].index[0]
    mod_events.loc[ii, 'peak_conc_time'] = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]
    mod_events.loc[ii, 'peak_load_time'] = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index[0]
    
# mod_events.loc[:, 'delta_time'] = mod_events.peaktime_load - mod_events.peaktime

In [34]:
mod_events.loc[:, 'delta_load_flow_time'] = pd.to_datetime(mod_events.peak_load_time) - pd.to_datetime(mod_events.peak_flow_time)
mod_events.loc[:, 'delta_conc_flow_time'] = pd.to_datetime(mod_events.peak_conc_time) - pd.to_datetime(mod_events.peak_flow_time)

In [40]:
mod_events.to_csv(f'{outpath}{mod_event_fn}.csv')

### Obs results

In [36]:
obs_event_fn = 'obs_storm_event_common'
obs_events = pd.read_csv(f'{outpath}{obs_event_fn}.csv', index_col = 'ID') 

In [37]:
# find the peak time of loads
for ii in obs_events.index:
    if ii < 38:
        df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, day_load_flow, timestep='d')
        clabel, llabel, qlabel = ['Concentration (mg/L)', 'Linear_Average_Load(t)', 'Flow(ML)']
    else:
        clabel, llabel, qlabel = ['126001A-NO3(mg/l)', 'Loads (kg)', 'Flow (ML)']
        df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, hour_load_flow, timestep='h')
    peaktime_load = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index[0]
    peaktime_conc = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]

    obs_events.loc[ii, 'peaktime_load'] = peaktime_load
    obs_events.loc[ii, 'peaktime_conc'] = peaktime_conc
    obs_events.loc[ii, 'peakflow'] = df_event.loc[:, qlabel].max()
    obs_events.loc[ii, 'peak_flow_time'] = df_event[df_event.loc[:, qlabel]==df_event.loc[:, qlabel].max()].index[0]
    obs_events.loc[ii, 'peak_conc_time'] = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]
    obs_events.loc[ii, 'peak_load_time'] = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index[0]
    
obs_events.loc[:, 'delta_load_flow_time'] = pd.to_datetime(obs_events.peak_load_time) - pd.to_datetime(obs_events.peak_flow_time)
obs_events.loc[:, 'delta_conc_flow_time'] = pd.to_datetime(obs_events.peak_conc_time) - pd.to_datetime(obs_events.peak_flow_time)

F:\Anaconda\envs\oed\lib\site-packages\pandas\core\tools\datetimes.py:129: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  unique_elements = set(islice(arg, check_count))
F:\Anaconda\envs\oed\lib\site-packages\pandas\core\tools\datetimes.py:129: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  unique_elements = set(islice(arg, check_count))
F:\Anaconda\envs\oed\lib\site-packages\pandas\core\tools\datetimes.py:129: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  unique_elements = set(islice(arg, check_count))
F:\Anaconda\envs\oed\lib\site-packages\pandas\core\tools\datetimes.py:129: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  unique_elements = set(islice(arg, check_count))


In [39]:
obs_events.to_csv(f'{outpath}{obs_event_fn}.csv')

## Variability of load-discharge ratio (seasonal average concentration)

### Obs results

In [22]:
time_ranges = [[f'{year}/7/1', f'{year}/10/1', f'{year+1}/1/1', f'{year+1}/4/1', f'{year+1}/7/1'] for year in range(2009, 2018)]
df_ratio = pd.DataFrame(index=[str(year) for year in range(2009, 2018)], columns = [1, 2, 3, 4])

In [23]:
for tt in time_ranges:
    for ii in range(len(tt) -1):
        start = pd.to_datetime(tt[ii])
        end = pd.to_datetime(tt[ii + 1]) - datetime.timedelta(days=1)   
        df = load_flow_loc([start, end], day_load_flow, timestep ='d')
        df_ratio.loc[tt[0][0:4], ii+1] = df.sum(axis=0)[0] / df.sum(axis=0)[2] * 1000

In [24]:
df_ratio.to_csv(f'{outpath}obs_seasonal_concentration.csv')

### Mod results

In [25]:
time_ranges = [[f'{year}/7/1', f'{year}/10/1', f'{year+1}/1/1', f'{year+1}/4/1', f'{year+1}/7/1'] for year in range(2009, 2018)]
df_ratio = pd.DataFrame(index=[str(year) for year in range(2009, 2018)], columns = [1, 2, 3, 4])

In [26]:
for tt in time_ranges:
    for ii in range(len(tt) -1):
        start = pd.to_datetime(tt[ii])
        end = pd.to_datetime(tt[ii + 1]) - datetime.timedelta(days=1)   
        df = load_flow_loc([start, end], mod_load_flow, timestep ='d')
        df_ratio.loc[tt[0][0:4], ii+1] = df.sum(axis=0)[0] / df.sum(axis=0)[2]

In [27]:
df_ratio.to_csv(f'{outpath}mod_seasonal_concentration.csv')

## Monthly loads

In [29]:
df_month = pd.DataFrame(columns = ['obs', 'mod'])

In [30]:
# calculate the monthly loads and flow
for y in range(2009, 2019):
    for m in range(1, 13):
        start = pd.to_datetime(f'{y}/{m}/1')
        if m == 12:
            end = pd.to_datetime(f'{y+1}/1/1') - datetime.timedelta(days=1) 
        else:
            end = pd.to_datetime(f'{y}/{m+1}/1') - datetime.timedelta(days=1)
            
        df_month.loc[f'{y}/{m}', 'obs'] = 1000 * load_flow_loc([start, end], day_load_flow, timestep ='d').sum(axis=0)[0]
        df_month.loc[f'{y}/{m}', 'mod'] = load_flow_loc([start, end], mod_load_flow, timestep ='d').sum(axis=0)[0]    
df_month = df_month[(df_month.obs != 0) & (df_month.loc[:, 'mod'] != 0)]
df_month.index.name = 'Month'

In [31]:
df_month.to_csv(f'{outpath}mod_obs_month.csv')

## Calculate the coefficients of variation for concentrations (CVC) and discharge (CVQ), their ratio (CVC:CVQ)

In [32]:
# define timeperiod
time_ranges = pd.to_datetime(['2009/7/1', '2018/6/30'])
df_cv = pd.DataFrame(columns=['obs', 'mod'], index=['cvc', 'cvq', 'cvl'])
start, end = time_ranges

In [33]:
# read obs time series of flow, loads and concentration
df_obs = load_flow_loc([start, end], day_load_flow, timestep ='d')
cv_all = (df_obs.std(axis=0) / df_obs.mean(axis=0))
cols = df_obs.columns
df_cv.loc[:, 'obs'] = [cv_all[cols[3]], cv_all[cols[2]], cv_all[cols[0]]]

In [34]:
# read mod time series of flow, loads and concentration
df_mod = load_flow_loc([start, end], mod_load_flow, timestep ='d')
cv_all = (df_mod.std(axis=0) / df_mod.mean(axis=0))
cols = df_mod.columns
df_cv.loc[:, 'mod'] = [cv_all[cols[1]], cv_all[cols[2]], cv_all[cols[0]]]

df_cv.loc['cq', :] = df_cv.loc['cvc', :] / df_cv.loc['cvq', :]
df_cv.loc['lq', :] = df_cv.loc['cvl', :] / df_cv.loc['cvq', :]

In [35]:
df_cv.to_csv(f'{outpath}cv_cql.csv')

## Linear regression of C-Q

In [3]:
# import necessary packages
from sklearn.metrics import r2_score
from utils.signatures import residual, nonlinear_fit
from utils.plotting import regression_plot
import lmfit

In [167]:
# define x and y
time_range = [['/7/1', '/10/1'], ['/10/1', '/1/1'], ['/1/1', '/4/1'], ['/4/1', '/7/1']]
cols = mod_load_flow.columns
# day_load_flow.loc[:, cols[0]] = day_load_flow.loc[:, cols[0]]*1000

In [171]:
x_dict, y_dict = {}, {}
k = 1
for tt in time_range:
    x, y = np.array([]), np.array([])
    for year in range(2009, 2019):
        start = pd.to_datetime(f'{year}{tt[0]}')
        if tt[1] == '/1/1':
            end = pd.to_datetime(f'{year+1}{tt[1]}') - datetime.timedelta(days=1)
        else:
            end = pd.to_datetime(f'{year}{tt[1]}') - datetime.timedelta(days=1)
        df_temp = load_flow_loc([start, end], mod_load_flow, timestep ='d')
        x = np.append(x, df_temp.values[:, 2])
        y = np.append(y, df_temp.values[:, 0])
    x_dict[f'{k}_x'] = x
    x_dict[f'{k}_y'] = y
    k += 1

In [179]:
# variables are x and y
coeff_regress = pd.DataFrame(columns = np.arange(1, 5), index=['R2', 'a', 'b', 'c'])
for k in range(1, 5):
    x = x_dict[f'{k}_x']
    y = x_dict[f'{k}_y']
    p = lmfit.Parameters()
    p.add_many(('a', 0.1, True, 0, 10), ('b', 2, True, 0, 2), ('c', 1, True, 0, 10))
    out1, out2, ci, trace = nonlinear_fit(p, residual, x, y, opti_method='differential_evolution')# lmfit, x=x_input, y=y_output,

    # compare coefficient of determination
    para_values = {}
    for param in ['a', 'b', 'c']: 
        para_values[param] = np.round(trace['a'][param][0], 4)
    y_mod = para_values['a'] * x ** para_values['b']+ para_values['c']
    r2 = r2_score(np.log(y), np.log(y_mod))
    abs_bias = np.abs(np.average(y_mod - y))
    rel_bias = abs_bias / np.average(y)
    coeff_regress.loc[:, k] = [r2, para_values['a'], para_values['b'], para_values['c']]

[[Variables]]
    a:  0.04437146 +/- 0.00512987 (11.56%) (init = 0.04437146)
    b:  1.28675687 +/- 0.03639772 (2.83%) (init = 1.286757)
    c:  0.00000000 +/- 3.2932e-11 (inf%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, b) = -0.915
    C(a, c) = -0.743
    C(b, c) =  0.672


C:\Users\qianw\anaconda3\envs\oed\lib\site-packages\lmfit\confidence.py:319: UserWarning: Bound reached with prob(c=0) = 0.0 < max(sigmas)
  warn(errmsg)


      95.45%    68.27%    _BEST_    68.27%    95.45%
 a:  -0.00226  -0.00108   0.04437  +0.00188  +0.00385
 b:  -0.00930  -0.00383   1.28676  +0.01333  +0.02660
 c:      -inf      -inf   0.00000  +0.00108  +0.00382
[[Variables]]
    a:  0.28381429 +/- 0.02335964 (8.23%) (init = 0.2838143)
    b:  1.09071015 +/- 0.02428153 (2.23%) (init = 1.09071)
    c:  0.00000000 +/- 8.8973e-11 (inf%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, b) = -0.752
    C(a, c) = -0.719
    C(b, c) =  0.531


C:\Users\qianw\anaconda3\envs\oed\lib\site-packages\lmfit\confidence.py:319: UserWarning: Bound reached with prob(c=0) = 0.0 < max(sigmas)
  warn(errmsg)


      95.45%    68.27%    _BEST_    68.27%    95.45%
 a:  -0.02341  -0.01192   0.28381  +0.01237  +0.02522
 b:  -0.02358  -0.01166   1.09071  +0.01140  +0.02252
 c:      -inf      -inf   0.00000  +0.00035  +0.00140
[[Variables]]
    a:  0.12792051 +/- 0.02013378 (15.74%) (init = 0.1279205)
    b:  1.02038065 +/- 0.02144549 (2.10%) (init = 1.020381)
    c:  6.51441502 +/- 0.44781624 (6.87%) (init = 6.514415)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, b) = -0.975
    C(a, c) = -0.628
    C(b, c) =  0.574
      95.45%    68.27%    _BEST_    68.27%    95.45%
 a:  -0.02670  -0.01326   0.12792  +0.01913  +0.04261
 b:  -0.03759  -0.01707   1.02038  +0.01171  +0.02700
 c:  -0.89112  -0.36047   6.51442  +0.32779  +0.74388
[[Variables]]
    a:  0.14737838 +/- 0.03075878 (20.87%) (init = 0.1477953)
    b:  1.08661110 +/- 0.04407610 (4.06%) (init = 1.086232)
    c:  0.56508922 +/- 0.20882216 (36.95%) (init = 0.5745331)
[[Correlations]] (unreported correlations are < 0.100)
   

In [181]:
coeff_regress.to_csv(outpath+'mod_cq_regress.csv')

## Calculate the variation of delivery ratio-surface

In [5]:
# read observations, modeled outputs with delivery ratio at 0 and 25 (%)
mod_fl_fn = 'DIN_sources.csv'
mod_drs0 = pd.read_csv(modpath + mod_fl_fn, index_col='Date')
mod_drs0.index = pd.to_datetime(mod_drs0.index, dayfirst=False)

In [6]:
arrays = [['surface', 'surface', 'surface', 'surface', 'seepage', 'seepage', 'seepage', 'seepage', 'DWC', 'DWC', 'DWC', 'DWC',
          'dwc_const', 'dwc_const', 'dwc_const', 'dwc_const', 'emc_const', 'emc_const', 'emc_const', 'emc_const'],
          ['median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean', 
           'median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean' ]]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples)

In [20]:
df_month = pd.DataFrame(columns=mod_drs0.columns)
# calculate the monthly loads and flow
for y in range(2009, 2019):
    for m in range(1, 13):
        start = pd.to_datetime(f'{y}/{m}/1')
        if m == 12:
#             import pdb; pdb.set_trace()
            end = pd.to_datetime(f'{y+1}/1/1') - datetime.timedelta(days=1) 
        else:
            end = pd.to_datetime(f'{y}/{m+1}/1') - datetime.timedelta(days=1)
#         
        df_month.loc[f'{y}/{m}', :] = load_flow_loc([start, end], mod_drs0, timestep ='d').sum(axis=0)
df_month = df_month[(df_month.loc[:, df_month.columns[0]] != 0)]
df_month.index.name = 'Month'

In [13]:
df_month.head()

,mod_all,mod_surface,mod_seepage,mod_DWC,mod_dwc,mod_emc,obs
Month,,,,,,,
2009/7,7.71861,0.753993,0.492195,1.64898,4.27022,0.553221,53.5864
2009/8,1.84695,0.0228792,0.233555,0.402991,1.14661,0.0409206,96.7843
2009/9,1.2901,0.00291682,0.227205,0.180846,0.467057,0.412074,88.9301
2009/10,0.469659,0.000538975,0.0561652,0.0576671,0.152037,0.20325,20.9705
2009/11,777.769,653.604,84.1427,0.878403,2.22915,36.9147,183.754


In [21]:
for col in df_month.columns:
    df_month[col+'_ratio'] = (df_month['obs'] - df_month['mod_all'] + df_month[col]) / df_month[col]
#     df_month['drs'] = (df_month['obs'] - df_month['mod_all'] + df_month[col]) / df_month[col]

In [22]:
df_month.head()

,mod_all,mod_surface,mod_seepage,mod_DWC,mod_dwc,mod_emc,obs,mod_all_ratio,mod_surface_ratio,mod_seepage_ratio,mod_DWC_ratio,mod_dwc_ratio,mod_emc_ratio,obs_ratio
Month,,,,,,,,,,,,,,
2009/7,7.71861,0.753993,0.492195,1.64898,4.27022,0.553221,53.5864,6.94249,61.8332,94.1903,28.8158,11.7413,83.9104,1.85596
2009/8,1.84695,0.0228792,0.233555,0.402991,1.14661,0.0409206,96.7843,52.4022,4150.51,407.487,236.582,83.7986,2321.04,1.98092
2009/9,1.2901,0.00291682,0.227205,0.180846,0.467057,0.412074,88.9301,68.9328,30047.4,386.732,485.611,188.643,213.68,1.98549
2009/10,0.469659,0.000538975,0.0561652,0.0576671,0.152037,0.20325,20.9705,44.6505,38037.7,366.01,356.503,135.841,101.865,1.9776
2009/11,777.769,653.604,84.1427,0.878403,2.22915,36.9147,183.754,0.236258,0.0911699,-6.05961,-675.244,-265.477,-15.0915,-2.23266


In [58]:
# df_concat.to_csv(outpath+'obs_mod_month.csv')

In [70]:
cols = ['median', 'min', 'max', 'mean_load_first']
x_list = [*np.arange(7, 13), *np.arange(1, 7)]
drs_stats = pd.DataFrame(columns=x_list, index=index)
drs_stats.index.name = 'month'
col_ratio = df_month.columns[8:13]
for col in col_ratio:
    k = list(col_ratio).index(col)
    for i in range(12):
        drs_stats.loc[index[(4 * k) : (4 * k + 3)], x_list[i]] = df_month[col][i::12].median(), df_month[col][i::12].min(), df_month[col][i::12].max()
        obs_mod = (df_month['obs'][i::12].mean() + df_month[df_month.columns[k+1]][i::12].mean() - df_month['mod_all'][i::12].mean())
        mod_mod0 = df_month[df_month.columns[k+1]][i::12].mean()
        drs_stats.loc[index[4 * k + 3], x_list[i]] = obs_mod / mod_mod0
drs_stats.to_csv(outpath+'DeliveryRatioSurface.csv')

In [71]:
drs_stats

7         8            9            10       11  \
surface   median    61.8332   57.3708      318.152      9.12839  4.70579   
          min      -1.58069   -7.3971     -18.1146     -23.8787 -18.2219   
          max       45586.7    635189  6.74423e+06  1.21484e+07  4234.49   
          mean   -0.0354917   4.86426      4.08439      1.45915  2.57306   
seepage   median    3.54406   21.4665      29.0852      4.12541  2.80973   
          min      0.194401  0.721546   -0.0598309   -0.0513656 -6.05961   
          max       98.6216   970.923      7136.16      38400.2  78.4866   
          mean     0.541971    5.7848      2.53332      1.30983  2.61828   
DWC       median    11.8215   261.788      640.365      410.542   539.58   
          min      -238.217  -67.0894     -772.634     -541.112 -675.244   
          max       468.971   660.948      1873.21      8197.88  3417.74   
          mean     -48.5025   288.958      336.216       62.436  147.828   
dwc_const median    5.03151   93.1716      285.858      155.636  215.005   
          min      -91.3949  -24.5127     -302.379     -171.063 -265.477   
          max       173.916   247.938      479.417         2009  1333.17   
          mean     -17.8047   103.506      135.058      29.5417  61.8501   
emc_const median    39.0629   101.081      88.9684      23.5649  32.9583   
          min      -3.70908  -9.81187      -102.49     -62.3364 -19.7794   
          max       341.315   2321.04      1242.91      5044.18  92.6453   
          mean     -1.46235   36.4883       22.602      4.31085  5.22092   

                        12       1         2         3         4         5   \
surface   median    2.7374  2.73682   1.07753 -0.390927  0.877783   21.1356   
          min     -1.24346  1.09246   -3.3863  -9.72023  -8.33126  -2.02975   
          max      7.40847   19.977   4.28071   0.86025    67.237    118.93   
          mean     1.68192  4.31499   1.01321 -0.317444 -0.410541   1.73949   
seepage   median   7.87321  4.95549   1.11148 -0.383699  0.725852   9.14394   
          min    -0.142813  1.04127 -0.360255  -3.78789  0.124273  0.208692   
          max      195.247  383.915   6.10918  0.819615   3.68493    24.334   
          mean     2.30851  5.56358   1.01246 -0.929288  0.540311   1.23592   
DWC       median   524.132  311.466   7.17943   -147.35  -5.71626   290.078   
          min     -1499.63  34.6432  -240.087  -484.332  -729.621   -275.54   
          max      5208.28  15904.4    1113.7  -14.7727   558.987   969.821   
          mean     111.339  507.691   1.91774  -143.429  -79.0047   47.0643   
dwc_const median   191.349  109.906   3.40882  -55.5585  -1.47459   113.315   
          min     -438.638  14.2514  -87.4078  -180.536  -261.142  -107.925   
          max      2157.08  6823.78   416.259  -7.00743   189.845   407.118   
          mean     44.0946  196.562   1.35248  -55.1981  -29.2796   18.9311   
emc_const median   14.4877  6.03583   1.11761 -0.618436  0.893577   49.8086   
          min     -19.3105  1.62953  -7.61821  -35.9762  -6.58799  -38.6478   
          max      158.801   345.22   15.9545  0.520823   48.2893   137.682   
          mean     2.48575  8.89182   1.01921 -0.447703 -0.745712   4.48914   

                        6   
surface   median   8.36317  
          min     0.747725  
          max       680.06  
          mean     2.39412  
seepage   median   4.87934  
          min     0.155011  
          max      146.778  
          mean     2.78637  
DWC       median   78.3427  
          min     -53.5327  
          max       421.79  
          mean     77.0525  
dwc_const median   31.0622  
          min     -22.9379  
          max      206.211  
          mean     30.9465  
emc_const median   31.2009  
          min     -2.34669  
          max      389.456  
          mean     12.2707